In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

In [2]:
df = pd.read_csv("titanic.csv")

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df = df.drop(["Name","Age","SibSp","Ticket","Cabin","Parch","Embarked"],axis = 1)

In [5]:
df.set_index("PassengerId", inplace=True)

In [6]:
df.describe()

,Survived,Pclass,Fare
count,891.000000,891.000000,891.000000
mean,0.383838,2.308642,32.204208
std,0.486592,0.836071,49.693429
min,0.000000,1.000000,0.000000
25%,0.000000,2.000000,7.910400
50%,0.000000,3.000000,14.454200
75%,1.000000,3.000000,31.000000
max,1.000000,3.000000,512.329200


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Fare      891 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 34.8+ KB


In [8]:
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Fare        0
dtype: int64

In [9]:
df['Sex'] = df['Sex'].astype('category').cat.codes

In [10]:
X = df.drop(["Survived"],axis = 1)
Y = df['Survived']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

In [12]:
X_train.shape

(623, 3)

In [13]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    gb = GradientBoostingClassifier(n_estimators=20, learning_rate = learning_rate, max_features=2, max_depth = 2, random_state = 0)
    gb.fit(X_train, y_train)
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb.score(X_test, y_test)))
    print()

Learning rate:  0.05
Accuracy score (training): 0.804
Accuracy score (validation): 0.746

Learning rate:  0.1
Accuracy score (training): 0.807
Accuracy score (validation): 0.746

Learning rate:  0.25
Accuracy score (training): 0.835
Accuracy score (validation): 0.791

Learning rate:  0.5
Accuracy score (training): 0.839
Accuracy score (validation): 0.787

Learning rate:  0.75
Accuracy score (training): 0.852
Accuracy score (validation): 0.806

Learning rate:  1
Accuracy score (training): 0.854
Accuracy score (validation): 0.765



In [15]:
gb = GradientBoostingClassifier(n_estimators=20, learning_rate = 0.5, max_features=2, max_depth = 2, random_state = 0)
gb.fit(X_train, y_train)
predictions = gb.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))
print()
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix:
[[142  11]
 [ 46  69]]

Classification Report
              precision    recall  f1-score   support

           0       0.76      0.93      0.83       153
           1       0.86      0.60      0.71       115

    accuracy                           0.79       268
   macro avg       0.81      0.76      0.77       268
weighted avg       0.80      0.79      0.78       268



In [16]:
y_scores_gb = gb.decision_function(X_test)
fpr_gb, tpr_gb, _ = roc_curve(y_test, y_scores_gb)
roc_auc_gb = auc(fpr_gb, tpr_gb)

print("Area under ROC curve = {:0.2f}".format(roc_auc_gb))

Area under ROC curve = 0.83
